In [1]:
import copy
import itertools

import numpy as np
import plotly.offline as pty
import plotly.graph_objs as ptygo
import plotly.grid_objs as ptygr
import plotly.tools as ptytools
import scipy.stats as st

import FilterPointCloud
import ProcessPlanePointCloud
import PLYObject

pty.init_notebook_mode(connected=True)

In [2]:
pcDataNear01 = ProcessPlanePointCloud.main('RealSense-D415-Data-01-Near/Laser/', {
    2.5: [
        'x<1.1447',
        'y<0.58',
    ]
})
%store pcDataNear01

Processing folder RealSense-D415-Plane-0.5m-1280x720-6-Laser
Filters: []
n = 4
e = [0.0012727441725343069, 0.0011975694788142934, 0.001200603294609284, 0.0011872591992506108]
d = [3599376.6029609237, 3673032.5532996035, 3674125.839804696, 3689355.2219764628]

Processing folder RealSense-D415-Plane-0.5m-640x480-6-Laser
Filters: []
n = 5
e = [0.0009796294877777807, 0.0009964415052368704, 0.0009896044632177158, 0.0009927092335213228, 0.0009845015563823933]
d = [1620227.3215344031, 1625838.3447183669, 1625583.8324894577, 1623391.8987467757, 1626158.7817603657]

Processing folder RealSense-D415-Plane-1.0m-1280x720-6-Laser
Filters: []
n = 5
e = [0.0041715402004331575, 0.003960588613275213, 0.003925371021466092, 0.0038850489694291007, 0.003886084727029298]
d = [871253.1860653976, 869398.9870511622, 866467.4519292689, 866021.1612865492, 870413.958580604]

Processing folder RealSense-D415-Plane-1.0m-640x480-6-Laser
Filters: []
n = 5
e = [0.0022913458993115625, 0.002204437930725687, 0.0022464650

In [3]:
pcDataNear02 = ProcessPlanePointCloud.main('RealSense-D415-Data-02-Near/Laser', {
    2.0: [
        'x>-1.03',
        'x<1.26',
    ],
    2.5: [
        'x>-0.7188',
        'x<1.26',
        'y>-0.88',
    ],
})
%store pcDataNear02

Processing folder RealSense-D415-Plane-0.5m-1280x720-6-Laser
Filters: []
n = 10
e = [0.0013949283153972532, 0.001425566796318968, 0.001432541879362279, 0.0014249105180535319, 0.0014376541124277438, 0.0014199190927026927, 0.0014235287504002284, 0.001417972333326971, 0.0014206605833859156, 0.0014177412176614428]
d = [3356355.665514503, 3405366.256603241, 3382880.305630357, 3396178.837125614, 3390912.9996983022, 3393509.276302911, 3405477.7994173346, 3386404.7944175247, 3390380.0750470236, 3395075.5521767964]

Processing folder RealSense-D415-Plane-0.5m-424x240-6-Laser
Filters: []
n = 10
e = [0.0011942174704500804, 0.0011666438089070988, 0.001156463877296269, 0.0011361359446997181, 0.0011375163845806955, 0.0011248273111395574, 0.0011350269871260679, 0.0011445629940927402, 0.0011324202834608076, 0.0011203401575615932]
d = [382867.725027025, 383337.8710544056, 383505.12333441636, 383339.09719899157, 382607.28795398225, 383328.1837409185, 383189.1568733527, 383657.00266254967, 383155.5853490

In [4]:
pcDataFar02 = ProcessPlanePointCloud.main('RealSense-D415-Data-02-Far/Laser', {})
%store pcDataFar02

Processing folder RealSense-D415-Plane-10.0m-1280x720-6-Laser
Filters: []
n = 10
e = [0.7733971665358543, 0.7758108570143196, 0.7830814561337377, 0.7923683657222009, 0.7839632170602483, 0.7855448405507924, 0.7853720224561366, 0.7767919847478462, 0.7825229903299087, 0.7736641785985531]
d = [20981.640723617154, 20374.789610638698, 19301.10248679749, 18839.68692918587, 19084.297434378634, 19184.23580579032, 20708.81557930165, 19408.495868250728, 19414.605668587974, 19207.56859854201]

Processing folder RealSense-D415-Plane-10.0m-640x480-6-Laser
Filters: []
n = 10
e = [0.919169557692313, 0.9499383241102616, 0.9621017537727948, 0.9469336677204843, 0.9484019708193643, 0.9422536551208209, 0.9329513486091747, 0.9219179395322048, 0.9374797912446827, 0.9405382708988239]
d = [8351.035411397033, 7967.383199581594, 8165.103794851284, 8245.882019558361, 7718.724939658396, 7839.189954871932, 7502.4175724122215, 8248.3778675438, 7951.451616431892, 7460.448372138121]

Processing folder RealSense-D415-P

In [5]:
def ptyColorMap(names):
    ptyColors = itertools.cycle(reversed(ptytools.DEFAULT_PLOTLY_COLORS))

    return dict(zip(names, ptyColors))

In [6]:
def createGraphObjectsFromPlaneData(data):
    goData = {}
    layoutData = {}
    
    colorMap = ptyColorMap(data.keys())
    
    measurandMap = {
        #'planeFitError': {
        #    'xaxis': {
        #        'title': 'Distance [m]',
        #    },
        #    'yaxis': {
        #        'title': 'Error [mm]',
        #    },
        #    'title': 'Plane Fit Error',
        #},
        'planeFitErrorStd': {
            'xaxis': {
                'title': 'Distance [m]',
            },
            'yaxis': {
                'title': 'Error STD [mm]',
                'range': [0, 7.3],
            },
            'title': 'Plane Fit Error Standard Deviation',
            'factor': 100,
        },
        'pointDensity': {
            'xaxis': {
                'title': 'Distance [m]',
            },
            'yaxis': {
                'title': 'Point Density [p / m²]',
                'tickformat': ',d',
                'hoverformat': ',.2f',
            },
            'title': 'Point Density',
            'factor': 1,
        },
    }
    
    resolutions = [*map(lambda res: 'x'.join([*map(str, res)]), sorted(map(lambda res: [*map(int, res.split('x'))], data.keys()), reverse=True))]
    
    for res in resolutions:
        showlegend = True
        
        for measurand in data[res]:
            if measurand not in measurandMap:
                continue
            
            distances = sorted(data[res][measurand].keys())
            errors = [np.array(data[res][measurand][dist]) * measurandMap[measurand]['factor'] for dist in distances]
            
            go = ptygo.Scatter(
                x=distances,
                y=[errorMeasurand.mean() for errorMeasurand in errors],
                error_y=dict(
                    type='data',
                    visible=True,
                    symmetric=True,
                    array=np.abs(np.array([st.t.interval(0.95, errorMeasurand.shape[0]-1, loc=0, scale=st.sem(errorMeasurand)) for errorMeasurand in errors])[:, 0]),
                    color=colorMap[res],
                ),
                name=res,
                legendgroup=res,
                showlegend=showlegend,
                line=dict(
                    color=colorMap[res],
                ),
            )
            
            if measurand not in goData:
                goData[measurand] = []
            
            goData[measurand].append(go)
            showlegend = False
            
            layoutData[measurand] = copy.deepcopy(measurandMap[measurand])
            
            if 'range' in layoutData[measurand]['yaxis']:
                layoutData[measurand]['yaxis']['range'][0] = min(0, *[data.min() for data in errors])
                layoutData[measurand]['yaxis']['range'][1] = max(7.3, *[data.max() for data in errors])
    
    return goData, layoutData

In [7]:
def plotPlaneData(data, title):
    go, lay = createGraphObjectsFromPlaneData(data)
    goKeys = [*go.keys()]
    rows = int(np.floor(np.sqrt(len(go))))
    cols = int(np.ceil(np.sqrt(len(go))))
    
    titles = tuple(lay[goKey]['title'] for goKey in goKeys)
    
    fig = ptytools.make_subplots(rows=rows, cols=cols, subplot_titles=titles, print_grid=False)
    fig['layout']['title'] = title
    
    for v in range(rows):
        for u in range(cols):
            i = v * rows + u
            
            if i >= len(go):
                break
            
            for t in go[goKeys[i]]:
                fig.append_trace(t, v + 1, u + 1)
            
            layout = lay[goKeys[i]]
            
            if 'xaxis' in layout:
                fig['layout']['xaxis{:d}'.format((v+1)*(rows-1)+u+1)].update(layout['xaxis'])
            
            if 'yaxis' in layout:
                fig['layout']['yaxis{:d}'.format((v+1)*(rows-1)+u+1)].update(layout['yaxis'])
    
    
    pty.iplot(fig)

In [8]:
%store -r pcDataNear01
plotPlaneData(pcDataNear01, 'Plan Fitting Near - Dataset 01')

In [9]:
%store -r pcDataNear02
plotPlaneData(pcDataNear02, 'Plan Fitting Near - Dataset 02')

In [10]:
%store -r pcDataFar02
plotPlaneData(pcDataFar02, 'Plan Fitting Far - Dataset 01')

In [ ]:
filt = FilterPointCloud.parseFilters([
    'x>-0.7188',
    'x<1.26',
    'y>-0.88',
])
ply = PLYObject.PLYObject(r'RealSense-D415-Data-02-Near\Laser\RealSense-D415-Plane-2.5m-424x240-6-Laser\1.pointcloud.ply')
fply = FilterPointCloud.filterPLYObject(ply, filt)

vert = ply.getVertices()
fvert = fply.getVertices()

data = [
    ptygo.Scatter3d(
        x=fvert[0],
        y=fvert[1],
        z=fvert[2],
        mode='markers',
        marker=dict(
            size=1,
            opacity=0.7,
        ),
    )
]
pty.iplot(data)

In [ ]:
data = [
    ptygo.Scatter3d(
        x=vert[0],
        y=vert[1],
        z=vert[2],
        mode='markers',
        marker=dict(
            size=1,
            opacity=0.7,
        ),
    )
]
pty.iplot(data)